In [ ]:
# Importações
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
# Carregar o dataset
marketing_data = pd.read_csv('/content/drive/MyDrive/AM/marketing_campaign.csv')

In [ ]:
# Transformações nos Dados
marketing_data['Age'] = date.today().year - marketing_data['Year_Birth']
marketing_data['Customer_Days'] = (pd.to_datetime('now', utc=True) - pd.to_datetime(marketing_data['Dt_Customer'], utc=True)).dt.days
marketing_data = pd.get_dummies(marketing_data, columns=['Marital_Status', 'Education'])


In [ ]:
# Remover colunas que não serão mais usadas
columns_to_drop = ['Year_Birth', 'Dt_Customer']
marketing_data.drop(columns=columns_to_drop, inplace=True)


In [ ]:
# Tratamento de outliers e dados faltantes para 'Income' e 'Age'
Q1_income = marketing_data['Income'].quantile(0.25)
Q3_income = marketing_data['Income'].quantile(0.75)
IQR_income = Q3_income - Q1_income
marketing_data = marketing_data[(marketing_data['Income'] < Q3_income + 1.5 * IQR_income) & (marketing_data['Income'] > Q1_income - 1.5 * IQR_income)]
Q1_age = marketing_data['Age'].quantile(0.25)
Q3_age = marketing_data['Age'].quantile(0.75)
IQR_age = Q3_age - Q1_age
marketing_data = marketing_data[(marketing_data['Age'] < Q3_age + 1.5 * IQR_age) & (marketing_data['Age'] > Q1_age - 1.5 * IQR_age)]


In [ ]:
# Dividir o dataset em características e rótulos
features = marketing_data.drop(['Response'], axis=1)
labels = marketing_data['Response']


In [ ]:
# Escalamento dos dados
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


In [ ]:
# Dividir o dataset em conjunto de treino e teste
X, X_valid, y, y_valid = train_test_split(features_scaled, labels, test_size=0.2, random_state=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
# Função para aplicar SMOTE em cada fold da validação cruzada
def cross_val_score_smote(model, X, y, cv=5, smote_proportion=0.4):
    skf = StratifiedKFold(n_splits=cv)
    scores = []

    for train_index, test_index in skf.split(X, y):
        X_train_fold, X_test_fold = X[train_index], X[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]

        # Contando as classes na dobra de treinamento
        class_counts = np.bincount(y_train_fold)
        minority_class = np.argmin(class_counts)
        majority_class = np.argmax(class_counts)

        # Aplicando SMOTE
        smote = SMOTE(sampling_strategy={minority_class: int(class_counts[majority_class] * smote_proportion)})
        X_train_fold_smote, y_train_fold_smote = smote.fit_resample(X_train_fold, y_train_fold)

        # Clonar o modelo para garantir que cada fold use um modelo "fresco"
        cloned_model = clone(model)
        cloned_model.fit(X_train_fold_smote, y_train_fold_smote)

        # Avaliar o modelo
        score = cloned_model.score(X_test_fold, y_test_fold)
        scores.append(score)

    return scores


In [ ]:
# Conversão para arrays NumPy
features_scaled_np = np.array(features_scaled)
labels_np = np.array(labels)

In [ ]:
# KNN
for k in [3, 5, 7]:
    # Opcional: Treinar o modelo no conjunto completo e avaliar no conjunto de teste
    # Isso é útil para ver o desempenho do modelo em dados não vistos
    # Aplicar SMOTE ao conjunto de treinamento completo
    smote = SMOTE(sampling_strategy=0.4, random_state=5)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    knn_model.fit(X_train_smote, y_train_smote)

    # Fazendo previsões no conjunto de teste
    predictions_knn = knn_model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, predictions_knn))
    print(classification_report(y_test, predictions_knn))

    knn_model = KNeighborsClassifier(n_neighbors=k)
    scores_knn = cross_val_score_smote(knn_model, features_scaled_np, labels_np, cv=5, smote_proportion=0.4)
    print(f"\nKNN Model (k={k}):")
    print("Média da Acurácia com CV:", np.mean(scores_knn))
    print("Desvio padrão da acurácia com CV:", np.std(scores_knn))


KNN Model (k=3):
Média da Acurácia com CV: 0.8263038548752834
Desvio padrão da acurácia com CV: 0.013948804080933982
Accuracy: 0.8226415094339623
              precision    recall  f1-score   support

           0       0.92      0.86      0.89       453
           1       0.42      0.58      0.49        77

    accuracy                           0.82       530
   macro avg       0.67      0.72      0.69       530
weighted avg       0.85      0.82      0.83       530


KNN Model (k=5):
Média da Acurácia com CV: 0.8258503401360544
Desvio padrão da acurácia com CV: 0.0100798688303369
Accuracy: 0.8358490566037736
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       453
           1       0.45      0.58      0.51        77

    accuracy                           0.84       530
   macro avg       0.69      0.73      0.70       530
weighted avg       0.86      0.84      0.84       530


KNN Model (k=7):
Média da Acurácia com CV: 0.83446712

In [ ]:

# Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train_smote, y_train_smote)
predictions_nb = nb_model.predict(X_test)
print("\nNaive Bayes Model:")
print("Accuracy:", accuracy_score(y_test, predictions_nb))
print(classification_report(y_test, predictions_nb))

# Usar a função cross_val_score_smote com arrays NumPy
scores_nb = cross_val_score_smote(nb_model, features_scaled_np, labels_np, cv=5, smote_proportion=0.4)
print("Média da Acurácia com CV:", np.mean(scores_nb))
print("Desvio padrão da acurácia com CV:", np.std(scores_nb))


Naive Bayes Model:
Accuracy: 0.7981132075471699
              precision    recall  f1-score   support

           0       0.93      0.83      0.88       453
           1       0.38      0.62      0.47        77

    accuracy                           0.80       530
   macro avg       0.65      0.73      0.67       530
weighted avg       0.85      0.80      0.82       530

Média da Acurácia com CV: 0.8045351473922903
Desvio padrão da acurácia com CV: 0.011597924582806741


In [ ]:
# Árvore de Decisão
dt_model = DecisionTreeClassifier(random_state=5)
dt_model.fit(X_train_smote, y_train_smote)
predictions_dt = dt_model.predict(X_test)
print("\nDecision Tree Model:")
print("Accuracy:", accuracy_score(y_test, predictions_dt))
print(classification_report(y_test, predictions_dt))

# Usar a função cross_val_score_smote para validação cruzada
scores_dt = cross_val_score_smote(dt_model, features_scaled_np, labels_np, cv=5, smote_proportion=0.4)
print("Média da Acurácia com CV:", np.mean(scores_dt))
print("Desvio padrão da acurácia com CV:", np.std(scores_dt))



Decision Tree Model:
Accuracy: 0.7981132075471699
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       453
           1       0.33      0.36      0.34        77

    accuracy                           0.80       530
   macro avg       0.61      0.62      0.61       530
weighted avg       0.81      0.80      0.80       530

Média da Acurácia com CV: 0.8222222222222222
Desvio padrão da acurácia com CV: 0.014667538892332181


In [ ]:
# RandomForest
rf_model = RandomForestClassifier(random_state=5)
rf_model.fit(X_train_smote, y_train_smote)
predictions_rf = rf_model.predict(X_test)
print("\nRandomForest Model:")
print("Accuracy:", accuracy_score(y_test, predictions_rf))
print(classification_report(y_test, predictions_rf))

# Usar a função cross_val_score_smote para validação cruzada
scores_rf = cross_val_score_smote(rf_model, features_scaled_np, labels_np, cv=5, smote_proportion=0.4)
print("Média da Acurácia com CV:", np.mean(scores_rf))
print("Desvio padrão da acurácia com CV:", np.std(scores_rf))


RandomForest Model:
Accuracy: 0.9
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       453
           1       0.80      0.42      0.55        77

    accuracy                           0.90       530
   macro avg       0.85      0.70      0.75       530
weighted avg       0.89      0.90      0.89       530

Média da Acurácia com CV: 0.8829931972789117
Desvio padrão da acurácia com CV: 0.0058431286742517494



#validação


In [ ]:
for k in [3, 5, 7]:
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X, y)
    # Fazendo previsões no conjunto de teste
    predictions_knn = knn_model.predict(X_valid)
    print("Accuracy:", accuracy_score(y_valid, predictions_knn))
    print(classification_report(y_valid, predictions_knn))

Accuracy: 0.8412698412698413
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       370
           1       0.52      0.24      0.33        71

    accuracy                           0.84       441
   macro avg       0.69      0.60      0.62       441
weighted avg       0.81      0.84      0.82       441

Accuracy: 0.8390022675736961
              precision    recall  f1-score   support

           0       0.86      0.96      0.91       370
           1       0.50      0.18      0.27        71

    accuracy                           0.84       441
   macro avg       0.68      0.57      0.59       441
weighted avg       0.80      0.84      0.81       441

Accuracy: 0.8526077097505669
              precision    recall  f1-score   support

           0       0.86      0.98      0.92       370
           1       0.64      0.20      0.30        71

    accuracy                           0.85       441
   macro avg       0.75      0.59      0.

In [ ]:
nb_model = GaussianNB()
nb_model.fit(X, y)
predictions_nb = nb_model.predict(X_valid)
print("\nNaive Bayes Model:")
print("Accuracy:", accuracy_score(y_valid, predictions_nb))
print(classification_report(y_valid, predictions_nb))


Naive Bayes Model:
Accuracy: 0.8321995464852607
              precision    recall  f1-score   support

           0       0.90      0.89      0.90       370
           1       0.48      0.51      0.49        71

    accuracy                           0.83       441
   macro avg       0.69      0.70      0.70       441
weighted avg       0.84      0.83      0.83       441



In [ ]:
dt_model = DecisionTreeClassifier(random_state=5)
dt_model.fit(X, y)
predictions_dt = dt_model.predict(X_valid)
print("\nDecision Tree Model:")
print("Accuracy:", accuracy_score(y_valid, predictions_dt))
print(classification_report(y_valid, predictions_dt))


Decision Tree Model:
Accuracy: 0.8095238095238095
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       370
           1       0.40      0.38      0.39        71

    accuracy                           0.81       441
   macro avg       0.64      0.64      0.64       441
weighted avg       0.81      0.81      0.81       441



In [ ]:
rf_model = RandomForestClassifier(random_state=5)
rf_model.fit(X, y)
predictions_rf = rf_model.predict(X_valid)
print("Accuracy:", accuracy_score(y_valid, predictions_rf))
print(classification_report(y_valid, predictions_rf))

Accuracy: 0.8571428571428571
              precision    recall  f1-score   support

           0       0.87      0.98      0.92       370
           1       0.65      0.24      0.35        71

    accuracy                           0.86       441
   macro avg       0.76      0.61      0.64       441
weighted avg       0.84      0.86      0.83       441

